# GAN using quick draw dataset of apple in npy format

#### select processing devices if you have a gpu(for nvidia)

In [ ]:
#import os
# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

#### Loading the dependencies

In [ ]:
# for data input and output:
import numpy as np
import os

# for deep learning: 
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Conv2D, BatchNormalization, Dropout, Flatten
from keras.layers import Activation, Reshape, Conv2DTranspose, UpSampling2D # new! 
from keras.optimizers import RMSprop

# for plotting: 
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
## Batch normalization is used to normalize the output since we have to many
## layers to we normalize the mean, std etc of our output
## Dropout is used to drop on layers or nodes in a layer to prevent overfitting
## Conv2DTranspose is opposite of Conv2D layer, it is deconvolution layer
## UpSampling2D is opposite of MaxPooling2D, In max pooling we decrease size
## of feature maps(or image), here we upscale them.

#### Load the data

In [ ]:
input_images = "apple.npy"
data = np.load(input_images)

In [ ]:
data = data/255
data = data.reshape(data.shape[0],28,28,1)
## resshaping image from (rows,784) to (rows,28,28,1) where 1 is channel for 
## grey

In [ ]:
img_w, img_h = data.shape[1:3] ## storing height and width i.e 28x28

In [ ]:
plt.imshow(data[3,:,:,0], cmap="Greys")

## Defining the Discriminator network
#### NOTE: width can be called as depth of network

In [ ]:
# where depth is the number of neurons and p is the dropout rate
# the dropout rate is 40% here on given training update
# these are hyper paramters that can be changed according to need
def discriminator_builder(depth=64,p=0.4):
    ## define inputs
    inputs = Input((img_w,img_h,1))
    ## 28x28x1
    ## Convolution layers:
        ## here depth maybe the number of feature maps where we 
        ## multiplied by 1 just in case we want to increase num of maps 
        ## we change the 1 to twice or depth paramter
        ## also 
    conv1 = Conv2D(depth*1, (5,5), strides=2, padding="same", activation='relu'
                   #input_shape=inputs
                   #we can use two ways of specifying input to this layer
                  )(inputs)
    ## adding a dropout too
    ## i think drop out drops some of the feature maps maybe?
    ## here output of this layer is 64 feature maps so we drop 40% maybe
    ## and give 60% to the next layer, not sure and need to confirm
    conv1 = Dropout(p)(conv1)
    
    ## having 128 maps now since depth=64 and * 2 = 128
    conv2 = Conv2D(depth*2, (5,5), strides=2, padding="same", 
                   activation='relu')(conv1)
    conv2 = Dropout(p)(conv2)
        
        
    conv3 = Conv2D(depth*4, (5,5), strides=2, padding="same", 
                   activation='relu')(conv2)
    conv3 = Dropout(p)(conv3)
    ## note we are manually connecting layers as you can see we give
    ## conv1 as input to conv2 layer and so on
    
    conv4 = Conv2D(depth*8, (5,5), strides=1, padding="same", 
                   activation='relu')(conv3)
    conv4 = Dropout(p)(conv4)
    
    ## time to flatten yo
    conv4 = Flatten()(conv4)
    
    ## output layer
    ## the output is one since it's binary whether the image is fake or real
    ## in this case whether the image has apple or not(true or false)
    output = Dense(1, activation="sigmoid")(conv4)
    
    ## Model definition
    ## input that we defined in beginning of function and output as 2 lines above
    model = Model(inputs=inputs,outputs=output)
    model.summary()
    
    return model

In [ ]:
discriminator = discriminator_builder()

In [ ]:
## decay is how learning rate slows down
## clipvalue kinda makes sure the parameter don't jump around too much
discriminator.compile(loss="binary_crossentropy",
                      optimizer=RMSprop(lr=0.0008,decay=6e-8, clipvalue=1.0),
                      metrics=['accuracy'])

## Defining the Generator network

In [ ]:
def generator_builder(z_dim=100,depth=64,p=0.4):
    
    # Define inputs
    inputs = Input((z_dim,))
    
    ## first dense layer
    ## here 7*7 can be seen as a shape like 28x28 so we construct an image
    ## from dense layer since it's opposite of discriminitor that converts
    ## an image to dense. Writing this for understanding
    dense1 = Dense(7*7*64)(inputs)
    
    ## we use batch normalization to maintain mean, variation of paramters
    ## through out the layers of our network, we can't risk variations as 
    ## training is being done
    ## momentum is like flexibility, we allow how much far variation can happen
    dense1 = BatchNormalization(momentum=0.9)(dense1) # default momentum for moving average is 0.99
    
    ## we are using activation after normalization rather before it
    dense1 = Activation(activation='relu')(dense1)
    
    ## we reshape our dense layer into to 3D 7x7x64 width,height,depth
    ## we reshape it so that we can feed it to discriminator
    dense1 = Reshape((7,7,64))(dense1)
    dense1 = Dropout(p)(dense1)
    
    ## now comes our deconvolutional part
    ## going to take 7x7 input and will up-sample to larger
    conv1 = UpSampling2D()(dense1)
    
    ## deconvolution layer reduces depth exponentially as we approach
    ## in network to discriminator.
    ## kernel_size should be same as above discrminitor kernel size 
    ## which is (5,5)
    ## activation is none since we will do it after batch normalization
    conv1 = Conv2DTranspose(int(depth/2), kernel_size=5, padding='same', activation=None,)(conv1)
    conv1 = BatchNormalization(momentum=0.9)(conv1)
    conv1 = Activation(activation='relu')(conv1)
    
    ## second deconv layer
    ## Conv2 takes Conv1 as input
    conv2 = UpSampling2D()(conv1)
    conv2 = Conv2DTranspose(int(depth/4), kernel_size=5, padding='same', activation=None,)(conv2)
    conv2 = BatchNormalization(momentum=0.9)(conv2)
    conv2 = Activation(activation='relu')(conv2)
    
    ## third deconv layer, we can have as many depending on situation
    ## Conv1 takes Conv2 as input
    ## not up-sampling here else size will be too large
    conv3 = Conv2DTranspose(int(depth/8), kernel_size=5, padding='same', activation=None,)(conv2)
    conv3 = BatchNormalization(momentum=0.9)(conv3)
    conv3 = Activation(activation='relu')(conv3)

    ## output layer:
    ## it is a convolutional layer which has single feature map output.
    ## we are using sigmoid since we want the feature map to have values between
    ## 0 and 1 where 0 would represent white and 1 is black
    ## since our discriminator takes values between 0 and 1 so we create matrix
    ## or img or feature map values between 0 and 1
    output = Conv2D(1, kernel_size=5, padding='same', activation='sigmoid')(conv3)
    
    ## model definition
    ## same way as discriminator
    model = Model(inputs=inputs, outputs=output)
    model.summary()
    
    return model

In [ ]:
generator = generator_builder()

 #### As you can see how we converted dense layer from 3136 to 7x7x64, then upsampling the size of 7x7 to 14x14 while reducing depth to 32 and then upsampling size of 14x14 to 28x28 while reducing depth to 16,8,1 now 28x28x1 is basically our input to discriminator.
 ---
 ## NOTE:
 #### we kept momentum of layers and size between discriminator and generator and successfully created 28x28x1 image/matrix from dense layers. #### It will be noise generation by generator at first but will update with time to improve the image it generates and get accurate as possible with time

---
## Create adversarial Network


In [ ]:
## z_dim is latent space
def adversarial_builder(z_dim=100):
        model = Sequential()
        model.add(generator)
        model.add(discriminator)
        model.compile(loss="binary_crossentropy",
                      optimizer=RMSprop(lr=0.004, decay=3e-8, clipvalue=1.0),
                      metrics=['accuracy']
                     )
        model.summary()
        
        return model

In [ ]:
adversarial_model = adversarial_builder()

### Training 

In [ ]:
## takes neural net as input and a boolean which the variable val
## the val represent whether the network should be trainable or not on 
## on our command
## The discriminator is used twice. One on it's own to detect fake and
## fake images. During that we want to be training the discriminator net
## second, we freeze the weights of the discriminator with this function
## so that we aren't changing weights of discriminator, instead we change the
## the weights of generator.


def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val
    

In [ ]:
def train(epochs=2000, batch=128):
    #we save accuracy as we train the nets
    d_metrics= [] # discriminator metrics
    a_metrics= [] # adversarial metrics
    
    running_d_lose = 0 # running loss discriminator
    running_d_acc = 0 # running acc discriminator
    
    running_a_lose = 0 # running loss advarsarial
    running_a_acc = 0 # running acc advarsarial
    
    for i in range(epochs):
        ## modular operation here
        if(i%100==0):
            print(i)
            
        real_imgs = np.reshape(data[np.random.choice(data.shape[0],batch,replace=False)],
                               (batch,28,28,1)
                              )
        ## images generated by generator, we give intial noise to generator
        ## before training our generator
        ## has 100 dimension latent space i.e the size
        ## should have same number of real and fake images so w provide batch
        fake_imgs = generator.predict(np.random.uniform(-1.0,1.0,size=[batch,100]))
        
        ## concatenate so we can give input to discriminator,
        ## x variable of training, we give fake and real images to 
        ## discriminator for training
        ## y is the label which tells which image are real and which are not
        ## so we tell discriminator which images are real and which are fake
        ## by using 0 or 1(1 for real)
        x= np.concatenate((real_imgs,fake_imgs))
        y = np.ones([2*batch,1])
        
        ## this selects rows starting from 128 to all next and assign them 0
        ## as we know x array values from 128 and so on are fakes so we use y
        ## those values as fake
        y[batch:,:] = 0
        
        make_trainable(discriminator, True)
        
        ## train_on_batch does the training of discriminator
        d_metrics.append(discriminator.train_on_batch(x,y))
        
        ## -1 indicate last value and 0 index in of loss which we will get 
        ## during training
        running_d_lose += d_metrics[-1][0] 
        running_d_acc += d_metrics[-1][1]
        
        
        make_trainable(discriminator, False)
        
        ## this time we label random noise as real images
        ## we use them to update the generator during training. the weights
        ## of discriminator are freezed during this time.
        ## we calculate the loss between this random noise and discriminator
        ## real images to update the generator.
        ## Note: this random noise is not used to train discrminator
        noise = np.random.uniform(-1.0,1.0,size=[batch,100])
        y = np.ones([batch,1])
        
        
        ## okay so we are giving a random batch of 128 size noise to input. that noise input for generator and the generator
        ## converts that noise into image/feature-map which is feeded into discriminator which the discriminator
        ## and it is shown to discriminator that the image is real. The discriminator determines whether image is fake/real
        ## calculates the loss but doesn't update itself since layers are frozen. the loss further is used to calculate
        ## the advsersairal loss meaning the whole network loss which is then used to update the network. since discriminator
        ## has frozen layers so it wont be updated, instead only the generator will be updated with the optimizer.
        ## the whole network loss will help make generator better in making accurate images
        a_metrics.append(adversarial_model.train_on_batch(noise,y))
        
        ## -1 indicate last value and 0 index in of loss which we will get 
        ## during training
        running_a_lose += a_metrics[-1][0] 
        running_a_acc += a_metrics[-1][1]
        
        if (i+1)%500 == 0:
            ## printing logs every 500 epochs
            print('Epoch #{}'.format(i+1))
            log_mesg = "%d: [D loss: %f, acc: %f]" % (i, running_d_loss/i, running_d_acc/i)
            log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, running_a_loss/i, running_a_acc/i)
            print(log_mesg)

            ## checking output of generator for a random noise
            noise = np.random.uniform(-1.0, 1.0, size=[16, 100])
            gen_imgs = generator.predict(noise)

            plt.figure(figsize=(5,5))

            ## plotting generator image to show
            for k in range(gen_imgs.shape[0]):
                plt.subplot(4, 4, k+1)
                plt.imshow(gen_imgs[k, :, :, 0], cmap='gray')
                plt.axis('off')
                
            plt.tight_layout()
            plt.show()
        return a_metrics, d_metrics

In [ ]:
a_metrics_complete, d_metrics_complete = train(epochs=1000)

In [ ]:

## ploting the loss and accuracy in dataframe way. a_metrics_complete contains 2D matrix, one part contains loss and other acc
## since there are multiple values so we ta
ke one row at time and take that rows 0 column which is the loss column in this case
ax = pd.DataFrame(
    {
        'Generator': [metric[0] for metric in a_metrics_complete],
        'Discriminator': [metric[0] for metric in d_metrics_complete],
    }
).plot(title='Training Loss', logy=True)
ax.set_xlabel("Epochs")
ax.set_ylabel("Loss")

In [ ]:
ax = pd.DataFrame(
    {
        'Generator': [metric[1] for metric in a_metrics_complete],
        'Discriminator': [metric[1] for metric in d_metrics_complete],
    }
).plot(title='Training Accuracy')
ax.set_xlabel("Epochs")
ax.set_ylabel("Accuracy")

## NOTE: I've done this code on google collab for faster results, It was good one. The comments made are used to clear up confusion or are used to make it easier to understand GAN  